In [1]:
!pip install -q datasets evaluate rouge_score accelerate --quiet
!pip install transformers --quiet
!pip install rouge_score --quiet
!pip install jiwer --quiet
!pip install git+https://github.com/google-research/bleurt.git --quiet

import transformers
from huggingface_hub import notebook_login
from evaluate import load
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
import accelerate
import torch
from torch.utils.data import Dataset

notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Pull Summary From Generated Dataset

In [2]:
dataset_test = load_dataset("jgibb/congress_117_bills_test_t5_summaries")["test"]
#dataset_test = load_dataset("jgibb/congress_117_bills_test_t5_summaries_billsum")["test"]

index = 14613

df = dataset_test.to_pandas()
df.set_index('index', inplace = True)

example = df.loc[index]['summary_bertsum_extractive_512_500']
example


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/377 [00:00<?, ? examples/s]

'This bill directs the Department of Defense (DOD) to reimburse the Department of Agriculture for the support provided by the Great Lakes Restoration Initiative and the Great Lakes Water Quality Agreement. The bill also authorizes the Department of Agriculture to transfer funds to the National Forest System (NFS) account.'